# Deep Hedging AI
### Recurrent Network: impact of using recurrent networks on hedging forward started options

Hans Buehler, January 2023


In [1]:
""" Slighly annoying: by default the SageMaker Python import directory does not include our git directory """
#!pip -q install cdxbasics "tensorflow>=2.10" "tensorflow_probability==0.14"
import os
p = os.getcwd()
dhn = "/deephedging/"
i = p.find(dhn)
if i!=-1:
    p = p[:i]
    import sys
    sys.path.append(p)
    print("SageMaker: added python path %s" % p)

In [2]:
print("Deep Hedging AI says hello ... ", end='')
from cdxbasics.config import Config
from cdxbasics.prettydict import PrettyOrderedDict as pdct
from deephedging.trainer import train
from deephedging.gym import VanillaDeepHedgingGym
from deephedging.world import SimpleWorld_Spot_ATM
import numpy as np
from cdxbasics.dynaplot import figure
from IPython.display import display, Markdown

# see print of the config below for numerous options
config = Config()
# world
config.world.steps = 20
#config.world.dt    = 1./50.
iReset = 10

nBins = 200

def fwd_start( spots ):
    terminal   = np.maximum( 1. * spots[:,iReset] - spots[:,-1], 0. )  # put
    return -terminal

config.world.samples = 20000
config.world.drift = 0.
config.world.cost_s = 0.
config.world.black_scholes = True
config.world.payoff = fwd_start
# gym
config.gym.objective.utility = "exp2"
config.gym.objective.lmbda = 5.
config.gym.agent.network.depth = 5
config.gym.agent.network.activation = "softplus"
config.gym.agent.recurrence.states.classic   = 1
config.gym.agent.recurrence.states.aggregate = 1
config.gym.agent.recurrence.states.past_repr = 1
config.gym.agent.recurrence.states.event     = 1
# trainer
config.trainer.train.optimizer.name = "adam"
config.trainer.train.optimizer.learning_rate = 0.001
config.trainer.train.optimizer.clipvalue = 0.9
config.trainer.train.optimizer.global_clipnorm = 1.
config.trainer.train.batch_size = config.world("samples")
config.trainer.train.epochs = 100
config.trainer.output_level = "quiet"
config.trainer.caching.mode = "on"
config.trainer.caching.epoch_freq = 100
config.trainer.visual.epoch_refresh = 20
config.trainer.visual.confidence_pcnt_lo = 0.25
config.trainer.visual.confidence_pcnt_hi = 0.75
config_recurrent = config.copy()

# create world
world  = SimpleWorld_Spot_ATM( config.world )
val_world  = world.clone(samples=world.nSamples//2)
gyms = pdct()

# plot
from deephedging.base import npCast, mean_bins
from cdxbasics.dynaplot import figure
import numpy as np

payoff = world.data.market.payoff
spots  = world.details.spot_all
fig = figure()
ax = fig.add_subplot()

ref  = spots[:,-1] / spots[:,iReset]
ixs  = np.argsort(ref)
xm   = mean_bins( ref[ixs], bins=nBins, weights=world.sample_weights, return_std = False )
m, s = mean_bins( payoff[ixs], bins=nBins, weights=world.sample_weights, return_std = True )

ax.plot( xm, m, color="blue")
ax.set_title("Payoff (over forward return)")
ax.set_xlabel("Spot/Reset")
ax.fill_between( xm, m-s,m+s, color="blue",alpha=0.2)

ax = fig.add_subplot()

ref  = spots[:,-1]
ixs  = np.argsort(ref)
xm   = mean_bins( ref[ixs], bins=nBins, weights=world.sample_weights, return_std = False )
m, s = mean_bins( payoff[ixs], bins=nBins, weights=world.sample_weights, return_std = True )

ax.plot( xm, m, color="blue")
ax.set_title("Payoff (over terminal return)")
ax.set_xlabel("Spot")
ax.fill_between( xm, m-s,m+s, color="blue",alpha=0.2)

fig.close()

Deep Hedging AI says hello ... Tensorflow version 2.10.0 running on 1 CPUs and 0 GPUs


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\hansb\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\hansb\AppData\Local\Temp\ipykernel_14928\1924258089.py", line 11, in <module>
    from deephedging.world import SimpleWorld_Spot_ATM
  File "C:\Users\hansb\iCloudDrive\Python3\deephedging\world.py", line 16, in <module>
    from scipy.stats import norm
  File "C:\Users\hansb\Anaconda3\lib\site-packages\scipy\stats\__init__.py", line 441, in <module>
    from .stats import *
  File "C:\Users\hansb\Anaconda3\lib\site-packages\scipy\stats\stats.py", line 43, in <module>
    from . import distributions
  File "C:\Users\hansb\Anaconda3\lib\site-packages\scipy\stats\distributions.py", line 10, in <module>
    from . import _continuous_distns
  File "C:\Users\hansb\Anaconda3\lib\site-pack

TypeError: object of type 'NoneType' has no len()

In [ ]:
import ray
ray.init(ignore_reinit_error=True)

display(Markdown("## Training with Ray in text mode"))

config1 = config_recurrent.copy()
config1.gym.agent.recurrence.states.classic   = 0
config1.gym.agent.recurrence.states.aggregate = 1
config1.gym.agent.recurrence.states.past_repr = 1
config1.gym.agent.recurrence.states.event     = 1
config1.label = "all"
config1.world.mark_done()

config2 = config_recurrent.copy()
config2.gym.agent.recurrence.states.classic   = 0
config2.gym.agent.recurrence.states.aggregate = 2
config2.gym.agent.recurrence.states.past_repr = 0
config2.gym.agent.recurrence.states.event     = 0
config2.label = "aggregate"
config2.world.mark_done()

config3 = config_recurrent.copy()
config3.gym.agent.recurrence.states.classic   = 0
config3.gym.agent.recurrence.states.aggregate = 0
config3.gym.agent.recurrence.states.past_repr = 2
config3.gym.agent.recurrence.states.event     = 0
config3.label = "past_repr"
config3.world.mark_done()

configs       = [ config1, config2, config3 ]
config_refs   = [ ray.put(c) for c in configs ]

def local_train(config):
    label = config("label", help="description")
    display(Markdown("## "+label))
    world  = SimpleWorld_Spot_ATM( config.world )
    val_world  = world.clone(samples=world.nSamples//2)
    gym = VanillaDeepHedgingGym( config.gym )
    train( gym=gym, world=world, val_world=val_world, config=config.trainer )
    config.done()
    return gym

@ray.remote
def remote_train(config ):
    from cdxbasics.config import Config
    from cdxbasics.prettydict import PrettyOrderedDict as pdct
    from deephedging.world import SimpleWorld_Spot_ATM
    from deephedging.trainer import train
    from deephedging.gym import VanillaDeepHedgingGym
    from IPython.display import display, Markdown
    local_train(config)

futures = [ remote_train.remote(c) for c in config_refs ]
results = ray.get(futures)

# results should now have been trained and cached
display(Markdown("## Loading cached data"))

gyms = { c['label'] : local_train(c) for c in configs }


In [ ]:
from deephedging.base import npCast, mean_bins
from cdxbasics.dynaplot import figure, colors_base
from scipy.stats import norm
import math as math
import numpy as np

spots      = world.details.spot_all
hedges     = world.data.market.hedges[:,:,0]
time_left  = world.data.features.per_step.time_left[0,:]
nSteps     = world.nSteps
nSamples   = world.nSamples
nBins     = 200

results = { k: npCast( gyms[k](world.tf_data) ) for k in gyms }
actions = { k: results[k]['actions'][:,:,0] for k in gyms }
deltas  = { k: np.cumsum( actions[k], axis=1) for k in gyms }

nBinSteps = 7
ixs       = np.linspace(0,nSteps,nBinSteps,endpoint=True,dtype=np.int32)
vol       = config.world.get_raw("rvol", 0.2)

pnls      = { k: np.zeros((nSamples,)) for k in gyms }
pnls['blackscholes'] = np.zeros((nSamples,))
prev_deltas_t  = { k: 0. for k in pnls }

keys_sorted = ['blackscholes'] + list( gyms.keys() )

fig = figure()
display(Markdown("## Forward Starting Option Hedging"))

for j in range(nSteps):
    ax_delta  = fig.add_subplot()
    ax_delta.set_ylim(-1.1,0.1)

    # sort by spot at j, and compute BS refernece
    spot_t    = spots[:,j]
    hedges_t  = hedges[:,j]   # S(T)-S(t_j)
    res_t     = time_left[j] 

    # BS
    # note that the 'drift' in the simulator is the statistical drift, not the risk-neutral drift.
    if j<iReset:
        ref_spot       = spot_t
        bsdelta_t      = ref_spot*0.
        ax_delta.set_title("Delta step %ld (before reset)" % j)
        ax_delta.set_xlabel("Spot")
    else:
        strike        = spots[:,iReset]
        d1            = ( np.log(spot_t/strike) +  0.5 * vol * vol  * res_t ) / math.sqrt( res_t * vol * vol )
        d2            = d1 - vol * math.sqrt( res_t )
        N1            = norm.cdf(d1)
        N2            = norm.cdf(d2)
        bsprice_t     = spot_t * N1 - strike * N2      # call
        bsprice_t     = bsprice_t + strike - spot_t    # C=P+S-K
        bsdelta_t     = N1
        bsdelta_t     = bsdelta_t - 1.
        ref_spot      = spot_t/strike
        ax_delta.set_title("Delta step %ld (after reset)" % j)
        ax_delta.set_xlabel("Spot/Strike")
        
    deltas_t  = { k: deltas[k][:,j] for k in deltas }
    deltas_t['blackscholes'] = bsdelta_t
    actions_t = { k: deltas_t[k] - prev_deltas_t[k] for k in deltas_t }
    dS_t      = spots[:,j+1] - spot_t
    pnls      = { k: pnls[k] + actions_t[k] * dS_t for k in pnls }

    # plot
    ixs       = np.argsort(ref_spot)
    ref_spot  = ref_spot[ixs]
    deltas_t  = { k: deltas_t[k][ixs] for k in deltas_t }
    
    x_mean          = mean_bins( ref_spot, bins=nBins, weights=world.sample_weights, return_std = False )
    colors          = colors_base()
    for k in keys_sorted:
        mean, std = mean_bins( deltas_t[k], bins=nBins, weights=world.sample_weights, return_std = True )
        color     = next(colors)
        ax_delta.plot( x_mean, mean, label=k, color=color)
        ax_delta.fill_between( x_mean, mean-std, mean+std, color=color, alpha=0.1 )

    ax_delta.legend()
    
fig.close()

# terminal
display(Markdown("## Terminal Profiles"))

spots    = world.details.spot_all
payoff   = world.data.market.payoff
payoff   = payoff - np.mean( payoff )

spot_ret = spots[:,-1] / spots[:,iReset]
ixs      = np.argsort(spot_ret)
spot_ret = spot_ret[ixs]
pnls     = { k : pnls[k][ixs] for k in pnls }
payoff   = payoff[ixs]

spot_mean = mean_bins( spot_ret, bins=nBins, weights=world.sample_weights, return_std = False )
payoff_mean, payoff_std = mean_bins( payoff, bins=nBins, weights=world.sample_weights, return_std = True )

fig = figure(col_size=6)
ax = fig.add_subplot()
ax.set_title("Terminal")
ax.set_xlabel("Spot/Strike")

colors = colors_base()
for k in keys_sorted:
    color = next(colors)
    mean, std = mean_bins( pnls[k], bins=nBins, weights=world.sample_weights, return_std = True )

    ax.plot( spot_mean, mean, label=k, color=color )
    ax.fill_between( spot_mean, mean-std, mean+std, color=color, alpha=0.1 )

color = next(colors)
ax.plot( spot_mean, -payoff_mean, label="-payoff", color=color)
ax.fill_between( spot_mean, -payoff_mean-payoff_std, -payoff_mean+payoff_std , color=color, alpha=0.1 )
ax.legend()
fig.close()